# DSA 2040 Practical Exam - Section 2, Task 3
## Classification - Customer Value Prediction

**Student:** Monaheng218  
**Date:** August 13, 2025  
**Total Marks:** 10

### Task Requirements:
1. Build classification models to predict customer value category
2. Compare multiple algorithms (Decision Tree, Random Forest, SVM, etc.)
3. Evaluate model performance using appropriate metrics
4. Feature importance analysis
5. Provide business insights for customer value prediction

In [ ]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Classification libraries imported successfully")
print("🎯 Ready for customer value prediction analysis")

In [ ]:
# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================

print("\n" + "="*60)
print("STEP 1: LOADING PREPROCESSED DATA")
print("="*60)

# Load the preprocessed classification dataset
X_classification = pd.read_csv('classification_features.csv')
y_classification = pd.read_csv('classification_target.csv')['CustomerValueCategory']
df_classification_full = pd.read_csv('classification_dataset_full.csv')

print(f"📊 Classification dataset loaded successfully")
print(f"Features shape: {X_classification.shape}")
print(f"Target shape: {y_classification.shape}")
print(f"Features: {list(X_classification.columns)}")

# Check target distribution
target_dist = pd.Series(y_classification).value_counts()
print(f"\n🎯 Target Variable Distribution:")
for class_label, count in target_dist.items():
    percentage = (count / len(y_classification)) * 100
    print(f"   Class {class_label}: {count} samples ({percentage:.1f}%)")

# Visualize target distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
target_dist.plot(kind='bar', alpha=0.8)
plt.title('Customer Value Category Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Customer Value Category')
plt.ylabel('Number of Customers')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
plt.pie(target_dist.values, labels=target_dist.index, autopct='%1.1f%%', startangle=90)
plt.title('Customer Value Category Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Feature statistics
print(f"\n📊 Feature Statistics:")
print(X_classification.describe())

# Check for missing values
missing_values = X_classification.isnull().sum().sum()
print(f"\n❌ Missing values: {missing_values}")

if missing_values == 0:
    print("✅ Data is clean and ready for classification")
else:
    print("⚠️ Handling missing values...")
    X_classification = X_classification.fillna(X_classification.mean())
    print("✅ Missing values handled")

In [ ]:
# =============================================================================
# DATA SPLITTING AND PREPARATION
# =============================================================================

print("\n" + "="*60)
print("STEP 2: DATA SPLITTING AND PREPARATION")
print("="*60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_classification, y_classification, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_classification
)

print(f"📊 Data splitting completed:")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Testing set: {X_test.shape[0]} samples")
print(f"   Features: {X_train.shape[1]}")

# Check class distribution in train/test sets
print(f"\n🎯 Class distribution in training set:")
train_dist = pd.Series(y_train).value_counts()
for class_label, count in train_dist.items():
    percentage = (count / len(y_train)) * 100
    print(f"   Class {class_label}: {count} samples ({percentage:.1f}%)")

print(f"\n🎯 Class distribution in testing set:")
test_dist = pd.Series(y_test).value_counts()
for class_label, count in test_dist.items():
    percentage = (count / len(y_test)) * 100
    print(f"   Class {class_label}: {count} samples ({percentage:.1f}%)")

# Feature correlation analysis
print(f"\n🔍 Feature Correlation Analysis:")
plt.figure(figsize=(10, 8))
correlation_matrix = X_train.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
           square=True, fmt='.2f')
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Check for highly correlated features
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append((
                correlation_matrix.columns[i], 
                correlation_matrix.columns[j], 
                correlation_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    print(f"⚠️ High correlation pairs found:")
    for feat1, feat2, corr in high_corr_pairs:
        print(f"   {feat1} - {feat2}: {corr:.3f}")
else:
    print(f"✅ No highly correlated features found (threshold: 0.8)")

print("\n✅ Data preparation completed successfully")

In [ ]:
# =============================================================================
# MODEL TRAINING AND COMPARISON
# =============================================================================

print("\n" + "="*60)
print("STEP 3: MODEL TRAINING AND COMPARISON")
print("="*60)

# Define classification models
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB()
}

# Train and evaluate models
results = {}
model_objects = {}

print("\n🔧 Training and evaluating models...")

for name, model in models.items():
    print(f"\n   📊 Training {name}...")
    
    # Train the model
    model.fit(X_train, y_train)
    model_objects[name] = model
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'CV Mean': cv_mean,
        'CV Std': cv_std,
        'Predictions': y_pred,
        'Probabilities': y_pred_proba
    }
    
    print(f"      ✅ Accuracy: {accuracy:.3f}, F1-Score: {f1:.3f}, CV: {cv_mean:.3f}±{cv_std:.3f}")

# Create results dataframe
results_df = pd.DataFrame({
    model: {
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score'],
        'CV Mean': metrics['CV Mean'],
        'CV Std': metrics['CV Std']
    }
    for model, metrics in results.items()
}).T

# Display results
print(f"\n📊 MODEL COMPARISON RESULTS:")
print("=" * 80)
print(results_df.round(3))

# Find best model
best_model_name = results_df['F1-Score'].idxmax()
best_model = model_objects[best_model_name]
best_f1 = results_df.loc[best_model_name, 'F1-Score']

print(f"\n🏆 Best performing model: {best_model_name}")
print(f"   🎯 F1-Score: {best_f1:.3f}")
print(f"   📊 Cross-validation: {results_df.loc[best_model_name, 'CV Mean']:.3f}±{results_df.loc[best_model_name, 'CV Std']:.3f}")

# Visualize model comparison
plt.figure(figsize=(16, 10))

# Performance metrics comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(results_df.index))
width = 0.2

plt.subplot(2, 2, 1)
for i, metric in enumerate(metrics_to_plot):
    plt.bar(x + i * width, results_df[metric], width, label=metric, alpha=0.8)

plt.title('Model Performance Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Models')
plt.ylabel('Score')
plt.xticks(x + width * 1.5, results_df.index, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Cross-validation scores
plt.subplot(2, 2, 2)
plt.errorbar(range(len(results_df)), results_df['CV Mean'], 
            yerr=results_df['CV Std'], marker='o', capsize=5, linewidth=2)
plt.title('Cross-Validation Scores with Error Bars', fontsize=14, fontweight='bold')
plt.xlabel('Models')
plt.ylabel('CV Accuracy')
plt.xticks(range(len(results_df)), results_df.index, rotation=45)
plt.grid(True, alpha=0.3)

# F1-Score ranking
plt.subplot(2, 2, 3)
f1_sorted = results_df.sort_values('F1-Score', ascending=True)
plt.barh(range(len(f1_sorted)), f1_sorted['F1-Score'], alpha=0.8)
plt.title('F1-Score Ranking', fontsize=14, fontweight='bold')
plt.xlabel('F1-Score')
plt.yticks(range(len(f1_sorted)), f1_sorted.index)
plt.grid(True, alpha=0.3)

# Accuracy vs F1-Score scatter
plt.subplot(2, 2, 4)
plt.scatter(results_df['Accuracy'], results_df['F1-Score'], s=100, alpha=0.7)
for i, model in enumerate(results_df.index):
    plt.annotate(model, (results_df['Accuracy'].iloc[i], results_df['F1-Score'].iloc[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)
plt.title('Accuracy vs F1-Score', fontsize=14, fontweight='bold')
plt.xlabel('Accuracy')
plt.ylabel('F1-Score')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Model training and comparison completed")

In [ ]:
# =============================================================================
# DETAILED EVALUATION OF BEST MODEL
# =============================================================================

print("\n" + "="*60)
print("STEP 4: DETAILED EVALUATION OF BEST MODEL")
print("="*60)

# Get predictions from best model
best_predictions = results[best_model_name]['Predictions']
best_probabilities = results[best_model_name]['Probabilities']

print(f"\n🔍 Detailed evaluation of {best_model_name}:")

# Classification report
print(f"\n📊 Classification Report:")
print(classification_report(y_test, best_predictions))

# Confusion matrix
cm = confusion_matrix(y_test, best_predictions)
class_names = sorted(y_test.unique())

plt.figure(figsize=(15, 5))

# Confusion Matrix
plt.subplot(1, 3, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=class_names, yticklabels=class_names)
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Confusion Matrix (Normalized)
plt.subplot(1, 3, 2)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
           xticklabels=class_names, yticklabels=class_names)
plt.title(f'Normalized Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# ROC Curves (for multi-class)
plt.subplot(1, 3, 3)
if best_probabilities is not None:
    # Convert to binary format for ROC analysis
    from sklearn.preprocessing import label_binarize
    from sklearn.metrics import roc_curve, auc
    from itertools import cycle
    
    # Binarize the output
    y_test_bin = label_binarize(y_test, classes=class_names)
    n_classes = y_test_bin.shape[1]
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], best_probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Plot ROC curves
    colors = cycle(['blue', 'red', 'green', 'orange', 'purple'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, linewidth=2,
                label=f'Class {class_names[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves - {best_model_name}', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
else:
    plt.text(0.5, 0.5, 'ROC curve not available\nfor this model', 
            ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('ROC Curve - Not Available', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Calculate per-class metrics
print(f"\n📊 Per-Class Performance:")
for i, class_name in enumerate(class_names):
    class_mask = y_test == class_name
    class_predictions = best_predictions == class_name
    
    tp = np.sum(class_mask & class_predictions)
    fp = np.sum(~class_mask & class_predictions)
    tn = np.sum(~class_mask & ~class_predictions)
    fn = np.sum(class_mask & ~class_predictions)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"   Class {class_name}:")
    print(f"      Precision: {precision:.3f}")
    print(f"      Recall: {recall:.3f}")
    print(f"      F1-Score: {f1:.3f}")
    print(f"      Support: {np.sum(class_mask)}")

print("\n✅ Detailed evaluation completed")

In [ ]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

print("\n" + "="*60)
print("STEP 5: FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Extract feature importance from best model
if hasattr(best_model, 'feature_importances_'):
    feature_importance = best_model.feature_importances_
    feature_names = X_train.columns
    
    # Create feature importance dataframe
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    print(f"\n📊 Feature Importance ({best_model_name}):")
    print(importance_df)
    
    # Visualize feature importance
    plt.figure(figsize=(12, 8))
    
    plt.subplot(1, 2, 1)
    plt.barh(range(len(importance_df)), importance_df['Importance'], alpha=0.8)
    plt.yticks(range(len(importance_df)), importance_df['Feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    cumulative_importance = importance_df['Importance'].cumsum()
    plt.plot(range(1, len(cumulative_importance) + 1), cumulative_importance, 
            marker='o', linewidth=2, markersize=6)
    plt.axhline(y=0.8, color='r', linestyle='--', alpha=0.7, label='80% threshold')
    plt.axhline(y=0.9, color='orange', linestyle='--', alpha=0.7, label='90% threshold')
    plt.xlabel('Number of Features')
    plt.ylabel('Cumulative Importance')
    plt.title('Cumulative Feature Importance', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Find features contributing to 80% and 90% of importance
    importance_80 = cumulative_importance <= 0.8
    importance_90 = cumulative_importance <= 0.9
    
    features_80 = importance_df[importance_80]['Feature'].tolist()
    features_90 = importance_df[importance_90]['Feature'].tolist()
    
    print(f"\n🎯 Key Feature Insights:")
    print(f"   📈 Top 3 most important features:")
    for i, (_, row) in enumerate(importance_df.head(3).iterrows()):
        print(f"      {i+1}. {row['Feature']}: {row['Importance']:.3f}")
    
    print(f"\n   📊 Features contributing to 80% of importance ({len(features_80)} features):")
    print(f"      {', '.join(features_80)}")
    
    print(f"\n   📊 Features contributing to 90% of importance ({len(features_90)} features):")
    print(f"      {', '.join(features_90)}")
    
elif hasattr(best_model, 'coef_'):
    # For linear models
    coef_importance = np.abs(best_model.coef_[0]) if best_model.coef_.ndim > 1 else np.abs(best_model.coef_)
    feature_names = X_train.columns
    
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': coef_importance
    }).sort_values('Importance', ascending=False)
    
    print(f"\n📊 Feature Importance (Coefficient Magnitude - {best_model_name}):")
    print(importance_df)
    
    # Visualize feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(importance_df)), importance_df['Importance'], alpha=0.8)
    plt.yticks(range(len(importance_df)), importance_df['Feature'])
    plt.xlabel('Feature Importance (|Coefficient|)')
    plt.title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
else:
    print(f"\n⚠️ Feature importance not available for {best_model_name}")
    
    # Use permutation importance as alternative
    from sklearn.inspection import permutation_importance
    
    print("\n🔍 Computing permutation importance...")
    perm_importance = permutation_importance(best_model, X_test, y_test, 
                                           n_repeats=10, random_state=42)
    
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': perm_importance.importances_mean,
        'Std': perm_importance.importances_std
    }).sort_values('Importance', ascending=False)
    
    print(f"\n📊 Permutation Feature Importance:")
    print(importance_df)
    
    # Visualize permutation importance
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(importance_df)), importance_df['Importance'], 
            xerr=importance_df['Std'], alpha=0.8)
    plt.yticks(range(len(importance_df)), importance_df['Feature'])
    plt.xlabel('Permutation Importance')
    plt.title(f'Permutation Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("\n✅ Feature importance analysis completed")

In [ ]:
# =============================================================================
# MODEL OPTIMIZATION (HYPERPARAMETER TUNING)
# =============================================================================

print("\n" + "="*60)
print("STEP 6: MODEL OPTIMIZATION (HYPERPARAMETER TUNING)")
print("="*60)

# Define parameter grids for top models
param_grids = {
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    }
}

# Select top 2 models for optimization
top_models = results_df.nlargest(2, 'F1-Score').index.tolist()
print(f"\n🎯 Optimizing top 2 models: {', '.join(top_models)}")

optimized_results = {}

for model_name in top_models:
    if model_name in param_grids:
        print(f"\n🔧 Optimizing {model_name}...")
        
        # Get the base model
        base_model = model_objects[model_name]
        
        # Perform grid search
        grid_search = GridSearchCV(
            base_model, 
            param_grids[model_name], 
            cv=5, 
            scoring='f1_weighted', 
            n_jobs=-1, 
            verbose=1
        )
        
        grid_search.fit(X_train, y_train)
        
        # Get optimized model
        optimized_model = grid_search.best_estimator_
        
        # Evaluate optimized model
        y_pred_opt = optimized_model.predict(X_test)
        
        opt_accuracy = accuracy_score(y_test, y_pred_opt)
        opt_f1 = f1_score(y_test, y_pred_opt, average='weighted')
        opt_cv_scores = cross_val_score(optimized_model, X_train, y_train, cv=5, scoring='accuracy')
        
        optimized_results[model_name] = {
            'Best Params': grid_search.best_params_,
            'Best CV Score': grid_search.best_score_,
            'Test Accuracy': opt_accuracy,
            'Test F1': opt_f1,
            'CV Mean': opt_cv_scores.mean(),
            'CV Std': opt_cv_scores.std(),
            'Model': optimized_model
        }
        
        print(f"   ✅ Best parameters: {grid_search.best_params_}")
        print(f"   📊 Best CV score: {grid_search.best_score_:.3f}")
        print(f"   🎯 Test F1-score: {opt_f1:.3f}")
        
        # Compare with original model
        original_f1 = results[model_name]['F1-Score']
        improvement = opt_f1 - original_f1
        print(f"   📈 Improvement over original: {improvement:+.3f}")
    else:
        print(f"\n⚠️ No parameter grid defined for {model_name}, skipping optimization")

# Display optimization results
if optimized_results:
    print(f"\n📊 OPTIMIZATION RESULTS:")
    print("=" * 80)
    
    opt_df = pd.DataFrame({
        model: {
            'Original F1': results[model]['F1-Score'],
            'Optimized F1': metrics['Test F1'],
            'Improvement': metrics['Test F1'] - results[model]['F1-Score'],
            'CV Score': metrics['Best CV Score']
        }
        for model, metrics in optimized_results.items()
    }).T
    
    print(opt_df.round(3))
    
    # Find best optimized model
    best_opt_model_name = opt_df['Optimized F1'].idxmax()
    best_opt_f1 = opt_df.loc[best_opt_model_name, 'Optimized F1']
    
    print(f"\n🏆 Best optimized model: {best_opt_model_name}")
    print(f"   🎯 Optimized F1-Score: {best_opt_f1:.3f}")
    print(f"   📈 Total improvement: {opt_df.loc[best_opt_model_name, 'Improvement']:+.3f}")
    
    # Update best model if optimization improved performance
    if best_opt_f1 > best_f1:
        best_model = optimized_results[best_opt_model_name]['Model']
        best_model_name = f"{best_opt_model_name} (Optimized)"
        print(f"\n✅ Updated best model to optimized version")
    else:
        print(f"\n📊 Original model performs better, keeping original")

print("\n✅ Model optimization completed")

In [ ]:
# =============================================================================
# BUSINESS INSIGHTS AND RECOMMENDATIONS
# =============================================================================

print("\n" + "="*70)
print("BUSINESS INSIGHTS AND CUSTOMER VALUE PREDICTION")
print("="*70)

# Generate predictions for the full dataset
full_predictions = best_model.predict(X_classification)
full_probabilities = best_model.predict_proba(X_classification) if hasattr(best_model, 'predict_proba') else None

# Add predictions to the original dataset
df_classification_full['PredictedValue'] = full_predictions
if full_probabilities is not None:
    for i, class_name in enumerate(sorted(y_classification.unique())):
        df_classification_full[f'Prob_{class_name}'] = full_probabilities[:, i]

# Analyze prediction accuracy by segment
print(f"\n🎯 PREDICTION ANALYSIS:")
print("=" * 50)

# Overall accuracy on full dataset
full_accuracy = accuracy_score(y_classification, full_predictions)
print(f"📊 Overall prediction accuracy: {full_accuracy:.3f}")

# Prediction distribution
pred_dist = pd.Series(full_predictions).value_counts()
actual_dist = pd.Series(y_classification).value_counts()

print(f"\n📊 Prediction vs Actual Distribution:")
comparison_df = pd.DataFrame({
    'Actual': actual_dist,
    'Predicted': pred_dist
}).fillna(0)
comparison_df['Difference'] = comparison_df['Predicted'] - comparison_df['Actual']
comparison_df['Difference_Pct'] = (comparison_df['Difference'] / comparison_df['Actual']) * 100
print(comparison_df)

# Model confidence analysis
if full_probabilities is not None:
    # Calculate prediction confidence (max probability)
    prediction_confidence = np.max(full_probabilities, axis=1)
    df_classification_full['Confidence'] = prediction_confidence
    
    print(f"\n🔍 Prediction Confidence Analysis:")
    print(f"   Average confidence: {prediction_confidence.mean():.3f}")
    print(f"   Confidence std: {prediction_confidence.std():.3f}")
    print(f"   High confidence predictions (>0.8): {(prediction_confidence > 0.8).sum()} ({(prediction_confidence > 0.8).mean()*100:.1f}%)")
    print(f"   Low confidence predictions (<0.6): {(prediction_confidence < 0.6).sum()} ({(prediction_confidence < 0.6).mean()*100:.1f}%)")

# Business insights based on feature importance and predictions
print(f"\n💡 BUSINESS INSIGHTS:")
print("=" * 50)

# Analyze which features are most predictive
print(f"\n🔍 Key Factors for Customer Value Prediction:")
if 'importance_df' in locals():
    top_features = importance_df.head(3)
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        feature_name = row['Feature']
        importance = row['Importance']
        
        print(f"\n{i}. {feature_name} (Importance: {importance:.3f})")
        
        # Analyze feature by customer value category
        feature_analysis = df_classification_full.groupby('CustomerValueCategory')[feature_name].agg(['mean', 'std']).round(3)
        
        for category in feature_analysis.index:
            mean_val = feature_analysis.loc[category, 'mean']
            std_val = feature_analysis.loc[category, 'std']
            print(f"   {category} customers: {mean_val:.3f} ± {std_val:.3f}")

# Customer value transition analysis
print(f"\n📈 CUSTOMER VALUE INSIGHTS:")
print("=" * 50)

# Analyze misclassifications to understand edge cases
test_indices = X_test.index
misclassified_mask = y_test != best_predictions
misclassified_indices = test_indices[misclassified_mask]

if len(misclassified_indices) > 0:
    print(f"\n🔍 Misclassification Analysis ({len(misclassified_indices)} cases):")
    
    misclass_analysis = pd.DataFrame({
        'Actual': y_test[misclassified_mask],
        'Predicted': best_predictions[misclassified_mask]
    })
    
    common_errors = misclass_analysis.groupby(['Actual', 'Predicted']).size().sort_values(ascending=False)
    print("   Most common misclassification patterns:")
    for (actual, predicted), count in common_errors.head(5).items():
        print(f"   • {actual} → {predicted}: {count} cases")

# Strategic recommendations
print(f"\n🚀 STRATEGIC RECOMMENDATIONS:")
print("=" * 50)

recommendations = [
    "📊 Use the model to predict customer value early in the relationship",
    "🎯 Focus marketing efforts on high-potential customers identified by the model",
    "📈 Monitor key predictive features to identify value improvement opportunities",
    "🔄 Regularly retrain the model with new customer data to maintain accuracy",
    "💡 Implement targeted interventions for customers predicted to be at risk of low value",
    "📧 Personalize customer communications based on predicted value category",
    "🎁 Design loyalty programs that align with predicted customer value segments"
]

for i, recommendation in enumerate(recommendations, 1):
    print(f"{i}. {recommendation}")

# Model deployment considerations
print(f"\n⚙️ MODEL DEPLOYMENT CONSIDERATIONS:")
print("=" * 50)
print(f"✅ Model Type: {best_model_name}")
print(f"✅ Accuracy: {results_df.loc[best_model_name.replace(' (Optimized)', ''), 'Accuracy']:.3f}")
print(f"✅ F1-Score: {best_f1:.3f}")
print(f"✅ Cross-validation stability: ±{results_df.loc[best_model_name.replace(' (Optimized)', ''), 'CV Std']:.3f}")
print(f"✅ Feature requirements: {len(X_classification.columns)} features")
print(f"✅ Training data size: {len(X_train)} samples")

# Save final model and results
import pickle

# Save the best model
with open('best_classification_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save predictions
df_classification_full.to_csv('customer_value_predictions.csv', index=False)

# Save model performance metrics
model_summary = {
    'best_model_name': best_model_name,
    'accuracy': full_accuracy,
    'f1_score': best_f1,
    'feature_importance': importance_df.to_dict('records') if 'importance_df' in locals() else None,
    'class_distribution': pred_dist.to_dict()
}

with open('classification_summary.pkl', 'wb') as f:
    pickle.dump(model_summary, f)

print(f"\n💾 Model and results saved:")
print(f"   • best_classification_model.pkl")
print(f"   • customer_value_predictions.csv")
print(f"   • classification_summary.pkl")

print("\n" + "="*70)
print("✅ CUSTOMER VALUE CLASSIFICATION COMPLETED SUCCESSFULLY")
print("Model trained, evaluated, and ready for deployment")
print("="*70)